# NTSB Aviation Accident Database Analysis

The following analysis looks at the NTSB Accident Database (linked here) and looks at accidents as a function of various database features, with the intention of identifying airplanes with a strong safety record that would be good to purchase for leasing purposes.

The features this analysis will focus on are:
*   Date of Accident
*   Airplane Make
*   Airplane Model
*   Aircraft Damage
*   Total Injuries
*   Total Fatalities
*   Phase of flight



## Data Cleaning

This particular analysis will look at the following features of the accident report.  

To begin, we will readin the data and clean up each of these features`

### Import the Data

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [109]:
#df = pd.read_csv("Data/Aviation_Data.csv")
df = pd.read_csv("https://raw.githubusercontent.com/jgoett2/AirplaneRecommendations/main/Data/Aviation_Data.csv")

<ipython-input-109-eb510d19be85>:2: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://raw.githubusercontent.com/jgoett2/AirplaneRecommendations/main/Data/Aviation_Data.csv")


### Date of Accident

Convert the string representation of Event.Date to a Pandas date-time object and store in same column.  Store the year in a new column "Year."

In [110]:
df["Event.Date"]=pd.to_datetime(df["Event.Date"], format="%Y-%m-%d")

df["Year"] = df["Event.Date"].dt.year

### Normalizing Make and Model

This analysis will concentrate on airplane models that are currently in production.  The dataframe models_of_interest holds data about these models.  
The database Make and Model values are updated to use consistent naming.

In [111]:
models_of_interest = pd.DataFrame([["Boeing","737",11762,"narrow-body"],
 ["Boeing","767",1311,"wide-body"], ["Airbus","A320",11471,"narrow-body"],
  ["Airbus","A340", 380,"wide-body"],["Boeing",'777',1729,"wide-body"],
   ["Airbus","A319",1501,"narrow-body"], ["Airbus",'A330',1601,"wide-body"],
    ["Airbus","A321",3145,"narrow-body"],["Boeing","787",1129,"wide-body"],
     ["Airbus","A350",602,"wide-body"], ["Airbus",'A220',335,"narrow-body"]],
                                  columns=["Make","Model","Produced","Type"])

Update the make and model columns in the dataframe to use the standardized names above.

In [112]:
def normalize(name, normalized_names):
    for x in normalized_names:
        if x.lower() in str(name).lower():
            return x
    return name

df["Make"]=df["Make"].map(lambda x:str(x))
df["Model"]=df["Model"].map(lambda x:str(x))

df["Make"]=df["Make"].map(lambda x:normalize(x,models_of_interest["Make"].unique()))
df["Model"]=df["Model"].map(lambda x:normalize(x,models_of_interest["Model"].unique()))

models_of_interest["Make"].unique()


array(['Boeing', 'Airbus'], dtype=object)

Add type of aircraft to dataframe

### Removing Extraneous Records

Airplane Makes and Models that are not of interest are removed from the data set.

In [124]:
filter = df["Make"].map(lambda x:x in models_of_interest["Make"].unique())
df = df[filter]

filter = df["Model"].map(lambda x:x in models_of_interest["Model"].unique())
df = df[filter]

Add aircraft type to dataframe.

In [125]:
models_of_interest = models_of_interest.set_index(["Make","Model"])

In [126]:
df["Airplane.Type"]=df.apply(lambda x: models_of_interest.loc[(x["Make"],x["Model"])]["Type"], axis=1)

In [123]:
models_of_interest=models_of_interest.reset_index()

In [127]:
features_of_interest=["Event.Date","Year","Make","Model","Airplane.Type","Aircraft.damage","Total.Fatal.Injuries",
                      "Total.Serious.Injuries","Broad.phase.of.flight"]

In [128]:
df=df[features_of_interest]

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1775 entries, 84 to 90338
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Date              1775 non-null   datetime64[ns]
 1   Year                    1775 non-null   float64       
 2   Make                    1775 non-null   object        
 3   Model                   1775 non-null   object        
 4   Airplane.Type           1775 non-null   object        
 5   Aircraft.damage         803 non-null    object        
 6   Total.Fatal.Injuries    1497 non-null   float64       
 7   Total.Serious.Injuries  1530 non-null   float64       
 8   Broad.phase.of.flight   396 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 138.7+ KB


In [130]:
df.to_csv("Airplane_data_clean.csv")

## Data Analysis
An analysis then looks at the number of incidents vs. airplane Make and Model.

### Accidents vs. Time

In [ ]:
count, years = np.histogram(df[df["Year"].isna()==False]["Year"],bins=range(1982,2024,1))
pd.DataFrame(count,index=years[0:41]).plot.bar()

### Injuries by Model

In [ ]:
for col in ["Total.Serious.Injuries","Total.Fatal.Injuries"]:
    df[col]=df[col].map(lambda x:0 if np.isnan(x) else int(x))

incidents = df.groupby(["Make","Model"]).count()["Total.Serious.Injuries"]
incidents_injuries=df[df["Total.Serious.Injuries"]>0].groupby(["Make","Model"]).count()["Total.Serious.Injuries"]

incidents_fatal=df[df["Total.Fatal.Injuries"]>0].groupby(["Make","Model"]).count()["Total.Fatal.Injuries"]
injury_total=df.groupby(["Make","Model"]).sum()[["Total.Serious.Injuries","Total.Fatal.Injuries"]]


In [ ]:
modelCompare=pd.concat([incidents,incidents_injuries,incidents_fatal,injury_total],axis=1)
modelCompare.columns=["Incidents-Any","Incidents-Injuries","Incidents-Fatalities","Total Injuries","Total Fatalities"]

for col in modelCompare.columns:
    modelCompare[col]=modelCompare[col].map(lambda x: 0 if np.isnan(x) else int(x))


produced = models_of_interest.set_index(["Make","Model"])
modelCompare = modelCompare.sort_values(["Make","Model"])
modelCompare = pd.concat([modelCompare,produced], axis=1)


In [ ]:
modelCompare["IncidentsRatio"]=modelCompare["Incidents-Any"]/modelCompare["Produced"]
modelCompare["IncidentsFatalitiesRatio"]=modelCompare["Incidents-Fatalities"]/modelCompare["Produced"]
modelCompare["IncidentsInjuriesRatio"]=modelCompare["Incidents-Injuries"]/modelCompare["Produced"]

In [ ]:
modelCompare[["Incidents-Any","Incidents-Injuries","Incidents-Fatalities","Total Injuries","Total Fatalities","Produced"]]


In [ ]:
fig, ax = plt.subplots()
modelCompare.plot.bar(y=["Incidents-Any","Incidents-Injuries","Incidents-Fatalities","Produced"], ax=ax)
ax.legend(["Incidents-Any","Incidents-Injuries","Incidents-Fatalities","Planes Produced"])
ax.set_xlabel("Model")
ax.set_ylabel("Count")
fig.savefig("fig1.png")

In [ ]:
fig, ax = plt.subplots()
modelCompare.plot.bar(y=["IncidentsRatio","IncidentsInjuriesRatio","IncidentsFatalitiesRatio"], ax=ax)
ax.legend(["Incidents:Planes Produced","Incidents-Injury:Planes Produced","Incidents-Fatality:Planes Produced"])
fig.savefig("fig2.png")

In [ ]:
df.info()

In [ ]:
df["Number.of.Engines"].value_counts()